In [106]:
import requests
from lxml import html
import json
import urllib
import xlrd
import boto
from boto.s3.key import Key
import csv
import datadotworld as dw


In [46]:

host = "https://esa.un.org"

base_url = host + "/unpd/wpp/Download"

sub_urls_to_crawl = '''
/SpecialAggregates/Ecological/
/Standard/Mortality/
/Standard/Errata/
/Standard/ASCII/
/SpecialAggregates/Political/
/Standard/Interpolated/
/Probabilistic/Population/
/Standard/Fertility/
/Other/MLT/
/Standard/Migration/
'''

In [81]:
def extract_sheets(filepath):
    xl_workbook = xlrd.open_workbook(filepath)
    out_data = {}
    for sheet in xl_workbook.sheet_names():
        in_context = False
        out_data[sheet] = []
        xl_sheet = xl_workbook.sheet_by_name(sheet)
        num_cols = xl_sheet.ncols   # Number of columns
        for row_idx in range(0, xl_sheet.nrows):
            row = []
            for col_idx in range(0, num_cols):  # Iterate through columns
                cell_obj = xl_sheet.cell(row_idx, col_idx)  # Get cell object by row, col 
                if cell_obj.value == 'Index':
                    in_context = True
                    
                if in_context:
                    row.append(cell_obj.value)
            if in_context:
                out_data[sheet].append(row)
                
    # remove empty
    empty = []
    for key, rows in out_data.items():
        if len(rows) == 0:
            empty.append(key)
    for e in empty:
        del out_data[e]
    return out_data

#out_data = extract_sheets("test_data.XLS")

In [89]:
def save_to_s3(orginal_file_name, key, rows):
    
    new_file_name = '{}_{}.csv'.format(orginal_file_name.replace(".", ""), key) 
    with open(new_file_name, 'w') as csvfile:
        spamwriter = csv.writer(csvfile)
        spamwriter.writerows(rows)
        
    s3 = boto.connect_s3()
    b = s3.get_bucket('brianray')
    k = Key(b)
    k.key = new_file_name
    k.set_contents_from_filename(new_file_name)
    k.set_acl('public-read')
    return k.generate_url(expires_in=0, query_auth=False)
    
#url = save_to_s3("test_data.XLS", "SOMETEST", [["test", "count"], ["a", 3], ["b", 4]])
#url


'https://brianray.s3.amazonaws.com:443/test_dataXLS_SOMETEST.csv'

In [125]:
def run_extraction(filepath):
    data_dict = extract_sheets(filepath)
    out_urls = {}
    for k, v in data_dict.items():
        out_url = save_to_s3(filepath, k, v)
        out_urls[out_url.split("/")[-1]] =  out_url
    return out_urls
    
    

In [102]:

def save(url):
    r = requests.get(url)
    file_name = url.split("/")[-1:][0]
    print(r.status_code)
    with open(file_name, 'wb') as f:
        f.write(r.content)
        
    return run_extraction(file_name)

#url = "https://esa.un.org/unpd/wpp/DVD/Files/1_Indicators%20%28Standard%29/EXCEL_FILES/1_Population/WPP2015_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.XLS"
#urls = save(url)
#urls

In [127]:
client = dw.api_client()

def save_to_dataworld(title, description, tags, link):
    data_args = dict(owner_id="brianray",
                     title=title[:30],
                     description=description[:120],
                     tags=tags,
                     license='Other',
                     visibility="PRIVATE", # "OPEN"
                     files=link)
    print(data_args)
    try:
        client.create_dataset(**data_args)
    except Exception as e:
        print(e)



In [129]:

def main():
    for sub in sub_urls_to_crawl.strip().split("\n"):
        url = "{}{}".format(base_url, sub)
        print(url)
        r = requests.get(url)
        print(r.status_code)
        tree = html.fromstring(r.content)
        json_str = tree.xpath("//script")[2].text.split("filesArray = ")[1].split("}];")[0] + "}]"
        for file_group_dict in json.loads(json_str):
            tags = ["un", "united nations"]
            tags += file_group_dict['MajorGroup'].split()
            tags.append(file_group_dict['MajorGroup'].replace(" ", "_"))
            path = "{}{}".format(host, urllib.parse.quote(file_group_dict['File1_Path']))
            file_paths = save(path)
            save_to_dataworld("United Nations {}".format(file_group_dict['File1_Title']),
                              file_group_dict['Description'],
                              tags,
                              file_paths)
                              
            break
        break

In [130]:
main()

https://esa.un.org/unpd/wpp/Download/SpecialAggregates/Ecological/
200
200
{'description': 'Total Population - Both Sexes. De facto population in a country, area or region as of 1 July of the year indicated. Figu', 'visibility': 'PRIVATE', 'tags': ['un', 'united nations', 'Population', 'indicators', 'Population_indicators'], 'files': {'WPP2015_POP_F01_1_TOTAL_POPULATION_BOTH_SEXESXLS_MEDIUM%20VARIANT.csv': 'https://brianray.s3.amazonaws.com:443/WPP2015_POP_F01_1_TOTAL_POPULATION_BOTH_SEXESXLS_MEDIUM%20VARIANT.csv', 'WPP2015_POP_F01_1_TOTAL_POPULATION_BOTH_SEXESXLS_ZERO-MIGRATION.csv': 'https://brianray.s3.amazonaws.com:443/WPP2015_POP_F01_1_TOTAL_POPULATION_BOTH_SEXESXLS_ZERO-MIGRATION.csv', 'WPP2015_POP_F01_1_TOTAL_POPULATION_BOTH_SEXESXLS_ESTIMATES.csv': 'https://brianray.s3.amazonaws.com:443/WPP2015_POP_F01_1_TOTAL_POPULATION_BOTH_SEXESXLS_ESTIMATES.csv', 'WPP2015_POP_F01_1_TOTAL_POPULATION_BOTH_SEXESXLS_HIGH%20VARIANT.csv': 'https://brianray.s3.amazonaws.com:443/WPP2015_POP_F01_1_T